## Imports and API

In [41]:
# Libraries
from openai import OpenAI
import requests
import feedparser
from dotenv import load_dotenv
import os

client = OpenAI()
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

## ArVix search agent

In [42]:
def search_arxiv(query, max_results=5):
    url = f'http://export.arxiv.org/api/query?search_query={query}&start=0&max_results={max_results}'
    feed = feedparser.parse(requests.get(url).text)
    
    results = []
    for entry in feed.entries:
        results.append({
            'title': entry.title,
            'summary': entry.summary,
            'link': entry.link
        })
    return results


## OpenAlex search agent

In [43]:
def search_openalex(query, max_results=5):
    url = f"https://api.openalex.org/works?search={query}&per-page={max_results}"
    response = requests.get(url)
    data = response.json()
    
    results = []
    for item in data.get('results', []):
        results.append({
            'title': item.get('display_name', 'No Title'),
            'summary': item.get('abstract_inverted_index', {}),
            'link': item.get('id', 'No Link')
        })
    return results


In [44]:
def merge_results(arxiv_results, openalex_results):
    return arxiv_results + openalex_results

## Summerizing agent

In [45]:
def summarize_papers(papers):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = f"Summarize each paper and highlight both opportunities and challenges. Include real-world examples or case studies when possible:\n{content}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [46]:
def summarize_papers_with_prompt(papers, custom_instruction):
    content = "\n\n".join([f"Title: {p['title']}\nSummary: {p['summary']}" for p in papers])
    prompt = f"{custom_instruction}\n\nPapers:\n{content}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


## Critic Agent

In [47]:
def critique_summary(summary_text):
    prompt = (
        "Review the following summary for bias, missing key points, or factual errors. "
        "Suggest improvements clearly:\n\n"
        f"{summary_text}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    critique = response.choices[0].message.content
    
    if any(keyword in critique.lower() for keyword in ["missing", "improve", "bias", "error"]):
        return critique, 'needs_rewrite'
    else:
        return critique, 'approved'


## Verifier agent

In [48]:
def verify_summary(summary_text):
    prompt = (
        "Fact-check the following summary. "
        "Highlight any factual inaccuracies or unsupported claims:\n\n"
        f"{summary_text}"
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content


In [49]:
def compose_final_answer(summary, critique, verification):
    return f"Summary:\n{summary}\n\nCritique:\n{critique}\n\nVerification Notes:\n{verification}"

In [50]:
# Example Query
query = "latest advancements in AI alignment"

# Step 1: Search
arxiv_results = search_arxiv(query, max_results=3)
openalex_results = search_openalex(query, max_results=3)
merged_results = merge_results(arxiv_results, openalex_results)

# Step 2: First Summarization
summary = summarize_papers(merged_results)

# Step 3: Critique the Summary
critique, status = critique_summary(summary)

# Step 4: If Critique Says 'Needs Rewrite' → Re-summarize with Refined Prompt
if status == 'needs_rewrite':
    refined_prompt = (
        "Summarize each research paper with focus on: "
        "- Main opportunities discussed\n"
        "- Key challenges or limitations\n"
        "- Any real-world applications or case studies mentioned\n"
        "Use clear, concise language."
    )
    # You need a function that accepts a custom prompt:
    summary = summarize_papers_with_prompt(merged_results, refined_prompt)
    critique, status = critique_summary(summary)  # Re-critique after re-summarization

# Step 5: Verification Step
verification = verify_summary(summary)

# Step 6: Final Answer
final_output = compose_final_answer(summary, critique, verification)
print(final_output)


Summary:
Title: Innovating for Tomorrow: The Convergence of SE and Green AI
- Main opportunities: Using foundation models to revolutionize software engineering processes and incorporating AI for automation.
- Key challenges: The environmental impact of using foundation models for software development and aligning practices with sustainability values.
- Real-world applications: Creating AI-enabled software systems with environmentally friendly practices.

Title: Reflection on Data Storytelling Tools in the Generative AI Era from the Human-AI Collaboration Perspective
- Main opportunities: Enhancing data storytelling with large-scale generative AI techniques and improving collaboration patterns.
- Key challenges: Reflecting on progress and identifying emerging collaboration patterns.
- Real-world applications: Human-creator + AI-assistant and AI-creator + human-reviewer collaboration patterns.

Title: WordDecipher: Enhancing Digital Workspace Communication with Explainable AI for Non-nat